In [60]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

In [10]:
bike_data = pd.read_csv('송파구1_최종.csv')

In [12]:
del bike_data['Unnamed: 0']

In [14]:
bike_data.head()

,자전거번호,대여일시,대여대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,이용시간,이용거리,...,반납소근처역,반납소Distance,대여일,대여시간,대여요일,반납일,반납시간,반납요일,최단거리,최단시간
0,SPB-48968,2022-06-01 00:26:14,4864,송파사거리,0,2022-06-01 00:33:19,1201,가락시장역 3번 출구,7,1170.00,...,가락시장,0.000727,2022-06-01,0,수,2022-06-01,0,수,1226.3,5
1,SPB-36076,2022-06-01 00:23:19,4474,가락현대 1차 11동 앞,0,2022-06-01 00:33:25,1201,가락시장역 3번 출구,10,1347.81,...,가락시장,0.000727,2022-06-01,0,수,2022-06-01,0,수,1078.9,4
2,SPB-58768,2022-06-01 00:20:49,4870,몽촌토성역 3번 출구,0,2022-06-01 00:42:55,1201,가락시장역 3번 출구,22,3440.00,...,가락시장,0.000727,2022-06-01,0,수,2022-06-01,0,수,3281.0,13
3,SPB-47296,2022-06-01 00:49:04,4462,가락중학교 앞,0,2022-06-01 00:58:23,1201,가락시장역 3번 출구,9,1330.00,...,가락시장,0.000727,2022-06-01,0,수,2022-06-01,0,수,1277.2,5
4,SPB-51146,2022-06-01 00:52:24,4462,가락중학교 앞,0,2022-06-01 00:58:28,1201,가락시장역 3번 출구,6,1216.22,...,가락시장,0.000727,2022-06-01,0,수,2022-06-01,0,수,1277.2,5


In [16]:
# 송파구의 데이터 중 필요한 열만 뽑아옵니다.
data = bike_data[['대여대여소번호','반납대여소번호','이용시간','이용거리','최단거리','최단시간']]

In [18]:
data.head()

,대여대여소번호,반납대여소번호,이용시간,이용거리,최단거리,최단시간
0,4864,1201,7,1170.00,1226.3,5
1,4474,1201,10,1347.81,1078.9,4
2,4870,1201,22,3440.00,3281.0,13
3,4462,1201,9,1330.00,1277.2,5
4,4462,1201,6,1216.22,1277.2,5


In [9]:
# 라벨링할 광고 번호를 아래의 범위 중 랜덤으로 설정합니다.
'''
이동수단의 데이터인 경우 0~29
운동의 데이터인 경우 30~59
여가의 데이터인 경우 60~89
'''

In [56]:
# 추가될 광고 인덱스를 담아주기 위한 배열입니다.
recommend_ad = ['']*len(data)

In [73]:
for i in tqdm(range(len(data))):
    # 1) 이동수단으로 분류할 데이터를 뽑아냅니다.
    # 이동수단 데이터 = ( 대여대여소번호 != 반납대여소번호) && ( 이용시간 <= 최단시간 * 1.5 && 이용거리 <= 최단거리 * 1.5 ) 
    diff_loca = (data['대여대여소번호'][i]!=data['반납대여소번호'][i])
    diff_time = (data['이용시간'][i] <= (data['최단시간'][i] * 1.5))
    diff_distance = (data['이용거리'][i] <= (data['이용거리'][i] * 1.5))
    if diff_loca and diff_time and diff_distance:
        recommend_ad[i] = random.randrange(0,30)
        continue
        
    # 2) 운동,여가로 분류할 데이터를 뽑아냅니다.
    # 시속 15km 이상으로 달리는 데이터는 운동, 15km 미만으로 달리는 데이터는 여가로 분류합니다.
    # 운동 데이터 = ( 이용거리/ 이용시간 >= 15)
    # 여가 데이터 = ( 이용거리/ 이용시간 < 15)
    km_distance = (data['이용거리'][i]/1000)
    hr_time = (data['이용시간'][i]/60)
    speed = km_distance/hr_time
    if speed >= 15:
        recommend_ad[i] = random.randrange(30,60)
    else:
        recommend_ad[i] = random.randrange(60,90)

100%|███████████████████████████████████████████████████████████████████████| 332846/332846 [00:15<00:00, 20889.13it/s]


In [77]:
# 광고 열을 추가해줍니다.
data['광고'] = recommend_ad

<ipython-input-77-9964c28f9200>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['광고'] = recommend_ad


In [78]:
data.head()

,대여대여소번호,반납대여소번호,이용시간,이용거리,최단거리,최단시간,광고
0,4864,1201,7,1170.00,1226.3,5,1
1,4474,1201,10,1347.81,1078.9,4,82
2,4870,1201,22,3440.00,3281.0,13,66
3,4462,1201,9,1330.00,1277.2,5,86
4,4462,1201,6,1216.22,1277.2,5,29
